In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms 

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:

device

# 데이터 불러오기

In [ ]:
cifar100 = torchvision.datasets.CIFAR100(root='./data', 
                                         train=True, 
                                         download=True)

# 데이터 분할하기

In [ ]:
# 7 : 1.5 : 1.5

In [ ]:
len(train_y), len(val_y), len(test_y)

# 데이터 전처리

In [ ]:
from torchvision import transforms

train_transform = transforms.Compose([
#     transforms.Resize((32, 32)), # 이미지 데이터의 사이즈가 제각각이므로 균일하게 잘라줘야 한다.
#     transforms.RandomHorizontalFlip(), # 대표적인 이미지 증강 기법
#     transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)), # 표준 정규분포로 만들어 주는 것 x-m/std
])

test_transform = transforms.Compose([
#     transforms.Resize(32, 32),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
])

# 데이터셋 & 데이터 로더 생성 

In [ ]:
from PIL import Image

class custom_dataset(torch.utils.data.Dataset):
    def __init__(self, x, y, transform):
        self.transform = transform
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        # numpy 배열을 PIL 이미지로 변환
        x = Image.fromarray(self.x[index])
        # 변환 적용
        x = self.transform(x)
        y = self.y[index]
        
        return x, y

# 모델 구성

In [ ]:
for x, y in train_dataloader:
    print(x.shape, y.shape)
    break

In [ ]:
class cnn(nn.Module):
    def __init__(self):
        super(cnn, self).__init__()
        
    def forward(self, x):
        
        return x

In [ ]:
model = cnn().to(device)
print(model)

In [ ]:
from sklearn.metrics import accuracy_score

def train_validate(model, train_dataloader, test_loader, optimizer, loss_func, epochs):
    train_losses = []
    train_accs = []
    validation_losses = []
    validation_accs = []

    for epoch in range(epochs):
        epoch_train_loss = 0.0
        epoch_train_acc = 0.0
        model.train()

        # 훈련 루프
        for data, target in train_dataloader:
            

            epoch_train_loss += loss.item()
            epoch_train_acc += acc

        # 에포크 당 평균 훈련 손실 및 정확도
        avg_train_loss = epoch_train_loss / len(train_dataloader)
        avg_train_acc = epoch_train_acc / len(train_dataloader)

        train_losses.append(avg_train_loss)
        train_accs.append(avg_train_acc)

        print(f"Epoch: {epoch+1}, Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f}")

        # 검증 루프
        model.eval()
        epoch_validation_loss = 0.0
        with torch.no_grad():
            for data, target in test_loader:
                
                epoch_validation_loss += loss.item()
        
        avg_validation_loss = epoch_validation_loss / len(test_loader)
        avg_validation_acc = accuracy_score(target.cpu(), y_pred.argmax(dim=1).detach().cpu())
        
        validation_losses.append(avg_validation_loss)
        validation_accs.append(avg_validation_acc)

        print(f"Validation Loss: {avg_validation_loss:.4f}, Validation Acc: {avg_validation_acc:.4f}")

    return train_losses, train_accs, validation_losses, validation_accs

In [ ]:
epochs=20

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Train")
plt.plot(validation_losses, label="Validation")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accs, label="Train")
plt.plot(validation_accs, label="Validation")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.legend()
plt.show()

In [ ]:
torch.save(model.state_dict(), "model.pth")

In [ ]:
model.load_state_dict(torch.load("model.pth"))

In [ ]:
def predict(model, dataloader):
    model.eval()  # 평가 모드로 전환
    all_preds = []

    with torch.no_grad():  # 그래디언트 계산 비활성화
        for data, _ in dataloader:
            data = data.to(device)  # 데이터를 장치로 이동
            
            # 모델 예측
            outputs = model(data)
            preds = outputs.argmax(dim=1)
            
            # 예측값을 리스트에 추가
            all_preds.extend(preds.cpu().numpy())

    return all_preds

In [ ]:
preds = predict(model, test_dataloader)

In [ ]:
confusion_matrix(test_y, preds)

In [ ]:
classification_report(test_y, preds)

In [ ]:
accuracy_score(test_y, preds)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print(f1_score(test_y, preds, average="macro"))
print(precision_score(test_y, preds, average="macro"))
print(recall_score(test_y, preds, average="macro"))